In [197]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
import re
import csv
from pykrx import stock
import urllib.parse as urlparse
from urllib.parse import parse_qs

In [198]:
driver = webdriver.Chrome('/Users/sion/Desktop/chromedriver')

url = 'https://finance.naver.com/news/news_search.naver'

driver.get(url=url)
time.sleep(2)

<ipython-input-198-35882b2aa0b3>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/sion/Desktop/chromedriver')


In [109]:
#시가총액 50위까지 종목 리스트 만들기
pdf = stock.get_index_portfolio_deposit_file("1028")

In [110]:
ticker_codes = pdf[:50]

In [111]:
for ticker in ticker_codes:
        ticker_names = stock.get_market_ticker_name(ticker)

In [199]:
ticker_names = ['삼성전자']
start_date = "2011-01-01"
end_date = "2011-01-20"

In [52]:
#형식 담을 딕셔너리
dic = {}

for ticker_name in ticker_names:

SyntaxError: unexpected EOF while parsing (<ipython-input-52-34e8b808b407>, line 4)

In [200]:
#뉴스가 최신순으로 나옴
searchbar = driver.find_element_by_xpath('//*[@id="contentarea_left"]/form/div/div/div/input[1]')
searchbar.clear()
searchbar.send_keys(ticker_names)
## 옵션 클릭
start = driver.find_element_by_name("stDateStart")
start.clear()
start.send_keys(start_date)


end = driver.find_element_by_name("stDateEnd")
end.clear()
end.send_keys(end_date)

search = driver.find_element_by_xpath('//*[@id="contentarea_left"]/form/div/div/div/input[2]')
search.click()
#페이지마다 내용 저장해보세
#뉴스제목
cur_url = driver.current_url

lastpage = driver.find_element_by_xpath('//*[@id="contentarea_left"]/table/tbody/tr/td[last()]/a')
lastpage.click()
lastpageindex = driver.find_element_by_xpath('//*[@id="contentarea_left"]/table/tbody/tr/td[last()]/a').text

if lastpageindex=="맨뒤":
    lastpageindex = 200
print(lastpageindex)
lastdate = driver.find_element_by_xpath('//*[@id="contentarea_left"]/div[2]/dl/dd[last()]/span[3]').text.split(' ')[0]
print(lastdate)

<ipython-input-200-13fb1ecefd65>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  searchbar = driver.find_element_by_xpath('//*[@id="contentarea_left"]/form/div/div/div/input[1]')
<ipython-input-200-13fb1ecefd65>:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  start = driver.find_element_by_name("stDateStart")
<ipython-input-200-13fb1ecefd65>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  end = driver.find_element_by_name("stDateEnd")
<ipython-input-200-13fb1ecefd65>:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = driver.find_element_by_xpath('//*[@id="contentarea_left"]/form/div/div/div/input[2]')
<ipython-input-200-13fb1ecefd65>:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  lastpage = driver.find_element_

200
2011-01-04


<ipython-input-200-13fb1ecefd65>:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  lastpageindex = driver.find_element_by_xpath('//*[@id="contentarea_left"]/table/tbody/tr/td[last()]/a').text
<ipython-input-200-13fb1ecefd65>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  lastdate = driver.find_element_by_xpath('//*[@id="contentarea_left"]/div[2]/dl/dd[last()]/span[3]').text.split(' ')[0]


In [201]:
while True:
    for i in range(1,int(lastpageindex)+1):
        tmp_page = cur_url + f'&page={i}'
        driver.get(tmp_page)
        time.sleep(1)

        articlecount = driver.find_elements_by_xpath('//*[@id="contentarea_left"]/div[2]/dl/dt')
        count= 1
        thumb=[]
        dd_cnt = 1 
        while(count<=len(articlecount)):
            print(f'\n{count}\n')
            test = driver.find_element_by_xpath(f'//*[@id="contentarea_left"]/div[2]/dl/dt[{count}]/*')
            #썸네일이 있으면 dt(count)는 썸네일이 가져감. 그리고 title은 dd로 계산됨 무조건 dt다음 dd가 있음
            #있는경우
            if test.find_elements_by_tag_name('img'):
                thumb = driver.find_element_by_xpath(f'//*[@id="contentarea_left"]/div[2]/dl/dt[{count}]/a/img').get_attribute('src')
                subject = driver.find_element_by_xpath(f'//*[@id="contentarea_left"]/div[2]/dl/dd[{dd_cnt}]/a')
                dd_cnt += 1
                title = subject.text
                articlelink = subject.get_attribute('href')

            #없는경우
            else:
                subject = driver.find_element_by_xpath(f'//*[@id="contentarea_left"]/div[2]/dl/dt[{count}]/a')
                title = subject.text
                articlelink = subject.get_attribute('href')

            info = driver.find_element_by_xpath(f'//*[@id="contentarea_left"]/div[2]/dl/dd[{dd_cnt}]').text
            date = info.split(" | ")[1]
            info = info.split(" | ")[0]

            try:
                press = info.split('.. ')[1]
                summary = info.split('.. ')[0] +'.'
            except IndexError:
                try:
                    press = info.split('다. ')[1]
                    summary = info.split('다. ')[0] +'다'
                except:
                    press = "None"
                    summary = "None"
                    pass


            dd_cnt += 1
            count += 1

            print(
            f'제목: {title}\n'
            f'작성일: {date}\n'
            f'요약: {summary}\n'
            f'언론사: {press}\n'
            f'썸네일: {thumb}\n'
            f'기사링크: {articlelink}'
            )
    lastdate = driver.find_element_by_xpath(f'//*[@id="contentarea_left"]/div[2]/dl/dd[last()]/span[3]').text.split(' ')[0]
    if lastdate == start_date:
        break
    else:
        end = driver.find_element_by_name("stDateEnd")
        end.clear()
        end.send_keys(lastdate)

        search = driver.find_element_by_xpath('//*[@id="contentarea_left"]/form/div/div/div/input[2]')
        search.click()
        
        cur_url = driver.current_url
        
        lastpage = driver.find_element_by_xpath('//*[@id="contentarea_left"]/table/tbody/tr/td[last()]/a')
        lastpage.click()
        lastpageindex = driver.find_element_by_xpath('//*[@id="contentarea_left"]/table/tbody/tr/td[last()]/a').text
        if lastpageindex=="맨뒤":
            lastpageindex = 200


<ipython-input-201-9a07eb1e9c1b>:7: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  articlecount = driver.find_elements_by_xpath('//*[@id="contentarea_left"]/div[2]/dl/dt')
<ipython-input-201-9a07eb1e9c1b>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  test = driver.find_element_by_xpath(f'//*[@id="contentarea_left"]/div[2]/dl/dt[{count}]/*')
/Users/sion/opt/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
<ipython-input-201-9a07eb1e9c1b>:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  subject = driver.find_element_by_xpath(f'//*[@id="contentarea_left"]/div[2]/dl/dt[{count}]/a')
<ipython-input-201-9a07eb1


1

제목: SK, 2013년까지 사회적기업 30곳 신설
작성일: 2011-01-20 21:45
요약: 회사 쪽은 “휘발유, 경유, 등유 등 수출액이 사상 최대인 25조원을 넘어서면서 삼성전자에 이어 국내에서는 두번째로 5년 누적수출액이 100조원을 넘어섰다”..
언론사: 한겨레
썸네일: []
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0002078255&office_id=028&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=1

2

제목: 풀린 돈 2000조…이 많은 돈은 다 어디로
작성일: 2011-01-20 21:34
요약: 국제신용평가사 무디스는 지난해 11월 삼성전자 등 국내 대기업들이 엄청난 현금을 보유하고 있다고 발표했다. 지난해 6월 말 현재 삼성전자는 18..
언론사: 경향신문
썸네일: https://imgnews.pstatic.net/image/thumb100/032/2011/01/20/2107300.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0002107300&office_id=032&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=1

3

제목: 삼성, ‘착한 기술’로 제3세계 돕는다
작성일: 2011-01-20 21:34
요약: ㆍ‘적정기술’ 개발·보급 협약 ㆍ돈벌이 무관한 R&D로 최초의 기업형 ‘재능기부’ 삼성전자가 앞선 연구개발(R&D) 기술력을 이용해 제3세계의 소외된 이웃 돕기에..
언론사: 경향신문
썸네일: https://imgnews.pstatic.net/image/thumb100/032/2011/01/20/2107300.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0002107303&office_id=032&mode=s

제목: 삼성전자 SSD 사전 공개 파티
작성일: 2011-01-20 17:55
요약: 삼성전자가 소비자용 ‘솔리드 스테이트 드라이브(SSD) 470 시리즈’ 출시를 앞둔 지난 19일 서울 서초동 소재 삼성전자 사옥 홍보관 딜라이트..
언론사: 파이낸셜뉴스
썸네일: https://imgnews.pstatic.net/image/thumb100/014/2011/01/20/2386626.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0002386626&office_id=014&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=2

5

제목: LGD 美 비지오 스마트TV 양산
작성일: 2011-01-20 17:53
요약: 북미 스마트TV 시장에서 1위인 삼성전자와 경쟁할 비지오의 스마트TV를 LG디스플레이의 관계사가 양산하게 되는 셈이다. 비아 플러스는 올해 1·4분기에 출시될..
언론사: 파이낸셜뉴스
썸네일: https://imgnews.pstatic.net/image/thumb100/014/2011/01/20/2386626.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0002386621&office_id=014&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=2

6

제목: SK이노베이션, 올해 1.5조 투자
작성일: 2011-01-20 17:52
요약: 특히 중국의 폭발적인 경유 수요 등으로 지난해 수출액은 25조8천억원을 기록했으며 최근 5년간 누적 수출액 역시 100조원을 넘어 삼성전자에 이..
언론사: 한국경제TV
썸네일: https://imgnews.pstatic.net/image/thumb100/014/2011/01/20/2386626.jpg
기사링크: https://finance.naver.com/news/news_read.na

제목: SK이노베이션, 누적 수출액 100조 뚫었다
작성일: 2011-01-20 17:06
요약: 삼성전자에 이어 국내 기업 중 두 번째다. SK이노베이션(대표 구자영)은 20일 2010년 실적발표에서 지난 한해 동안 매출 43조 8675억 원, 영업이익 1조 7068..
언론사: 전자신문
썸네일: https://imgnews.pstatic.net/image/thumb100/009/2011/01/20/2395578.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0002104862&office_id=030&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=3

11

제목: 이석채 KT 회장 "재벌기업과의 싸움이 가장 힘들었다"
작성일: 2011-01-20 17:04
요약: 그는 특정 업체를 지칭하지는 않았지만, 업계에서는 삼성전자와 SK텔레콤인 것으로 풀이하고 있다. KT와 삼성전자는 애플 아이폰 도입 이후 관계가..
언론사: 뉴시스
썸네일: https://imgnews.pstatic.net/image/thumb100/003/2011/01/20/3652083.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0003652083&office_id=003&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=3

12

제목: [2011 삼성 승부수] 삼성증권
작성일: 2011-01-20 17:03
요약: 새로운 차원의 증권회사, 금융의 ‘삼성전자’로 성장하고 있다는 평가다. 삼성증권은 지난해 11월 증권 업종 대장주로 복귀했으며 시가총액도 6조원..
언론사: 서울경제
썸네일: https://imgnews.pstatic.net/image/thumb100/011/2011/01/20/2126609.jpg
기사링크: https://finance.naver.c

제목: 특허청 · 삼성전자, 최빈국 기술 지원 `한 뜻`
작성일: 2011-01-20 16:03
요약: 특허청과 삼성전자가 축적된 기술 및 연구인력을 바탕으로 최빈국과 개발도상국이 필요로 하는 적정기술을 개발하고 보급하는데 함께 나선다. 이수원 특허청장과..
언론사: 전자신문
썸네일: https://imgnews.pstatic.net/image/thumb100/011/2011/01/20/2126560.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0002104834&office_id=030&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=4

16

제목: 삼성전자-특허청, '착한기술'로 나눔 실천한다
작성일: 2011-01-20 16:03
요약: 삼성전자가 특허청과 함께 '적정기술' 개발과 보급에 협력하기로 했다. 한민호 삼성전자 디지털시티센터장과 이수원 특허청장은 20일 오후..
언론사: 아이뉴스24
썸네일: https://imgnews.pstatic.net/image/thumb100/031/2011/01/20/214294.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0000214294&office_id=031&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=4

17

제목: 삼성전자-특허청, 기술을 통한 글로벌 나눔에 동참
작성일: 2011-01-20 16:02
요약: [CBS 삼성전자가 특허청과 함께 '적정기술' 개발과 보급을 위해 협력하기로 했다. 한민호 삼성전자 디지털시티센터장과 이수원 특허청장은 20일..
언론사: 노컷뉴스
썸네일: https://imgnews.pstatic.net/image/thumb100/031/2011/01/20/214294.jpg
기사링크: https://finance.naver.com/news/ne

...

제목: 막 오른 3세 경영ㆍ미래사업 기선잡기 … 재계 '지각변동' 예고
작성일: 2011-01-02 18:31
요약: (2) 출사표 던진 3세 경영인 실적은 이재용 삼성전자 사장과 이부진 호텔신라 · 에버랜드 사장 등 삼성가(家) 3세들은 작년 말 일제히 승진했..
언론사: 한국경제
썸네일: https://imgnews.pstatic.net/image/thumb100/015/2011/01/02/2364454.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0002364454&office_id=015&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=32

14

제목: 無線 충전기술 첫 적용 '이건희 3D 안경'
작성일: 2011-01-02 18:31
요약: 삼성전자가 무선 충전기술을 적용한 3D TV용 안경을 개발해 6일 미국 라스베이거스에서 개막하는 CES에서 공개한다. 오스트리아의 안경 브랜드 ..
언론사: 한국경제
썸네일: https://imgnews.pstatic.net/image/thumb100/015/2011/01/02/2364455.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0002364455&office_id=015&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=32

15

제목: 대학생이 만나고 싶은 CEO … 정준양 회장·김정태 행장
작성일: 2011-01-02 18:31
요약: 한경비즈니스가 전국 대학생 659명을 대상으로 '가장 만나고 싶은 CEO'를 조사한 결과 제조 부문에서는 정준양 포스코 회장(25.78%)과 최..
언론사: 한국경제
썸네일: https://imgnews.pstatic.net/image/thumb100/015/2011/01/02/2364458.jpg
기사링크: https://finance.n

제목: [2011 재테크] 해외특파원이 본 글로벌 인기상품
작성일: 2011-01-02 16:58
요약: 뉴욕타임스(NYT)지에 따르면 MS 태블릿PC의 생산은 삼성전자와 델 컴퓨터를 통해 이뤄질 예정이며 애플과 구글 태블릿PC와는 달리 슬라이드형 ..
언론사: 파이낸셜뉴스
썸네일: https://imgnews.pstatic.net/image/thumb100/014/2011/01/02/2378187.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0002378187&office_id=014&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=33

17

제목: [2011 재테크] 올 증시 달굴 유망종목 10개
작성일: 2011-01-02 16:55
요약: 삼성전자가 증권사들이 뽑은 2011년 최고의 투자 유망 종목으로 뽑혔다. 삼성전자는 삼성증권, 동양종합금융증권, 우리투자증권, 대신증권 등에서 ..
언론사: 파이낸셜뉴스
썸네일: https://imgnews.pstatic.net/image/thumb100/014/2011/01/02/2378186.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0002378186&office_id=014&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=33

18

제목: 사자성어로 풀어본 주요 그룹 새해 전략
작성일: 2011-01-02 16:33
요약: 이와 달리 영업이익 17조원을 올린 삼성전자는 자만을 경계하는 교병필패를 여전히 마음속에 새기는 모습이다. 2일 재계에 따르면 주요 그룹 임직원..
언론사: 서울경제
썸네일: https://imgnews.pstatic.net/image/thumb100/011/2011/01/02/2122138.jpg
기사링크: https://finance.naver.com/n

제목: [CES2011]삼성전자 "명품 3D 안경 공개"
작성일: 2011-01-02 11:22
요약: 【서울=뉴시스】 삼성전자는 세계적인 명품 안경 브랜드 '실루엣'과 제휴를 통해 세계 초경량 셔터글래스 3D 안경을 오는 6~9일 나흘간 미국 라..
언론사: 뉴시스
썸네일: https://imgnews.pstatic.net/image/thumb100/003/2011/01/02/3620336.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0003620336&office_id=003&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=34

1

제목: [CES2011]삼성전자 "명품 3D 안경 공개"
작성일: 2011-01-02 11:22
요약: 【서울=뉴시스】 삼성전자는 세계적인 명품 안경 브랜드 '실루엣'과 제휴를 통해 세계 초경량 셔터글래스 3D 안경을 오는 6~9일 나흘간 미국 라..
언론사: 뉴시스
썸네일: https://imgnews.pstatic.net/image/thumb100/003/2011/01/02/3620337.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0003620337&office_id=003&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=35

2

제목: [CES2011]삼성전자 "명품 3D 안경 공개"
작성일: 2011-01-02 11:22
요약: 【서울=뉴시스】 삼성전자는 세계적인 명품 안경 브랜드 '실루엣'과 제휴를 통해 세계 초경량 셔터글래스 3D 안경을 오는 6~9일 나흘간 미국 라..
언론사: 뉴시스
썸네일: https://imgnews.pstatic.net/image/thumb100/003/2011/01/02/3620334.jpg
기사링크: https://finance.naver.com/news/

제목: 송파, 영등포, 강남 등 전셋값, 1년간 10% 이상 상승
작성일: 2011-01-02 10:30
요약: 화성(0.7%)은 삼성전자 등 인근 산업단지 협력업체 등의 근로자수요, 인근에 비해 상대적으로 가격이 저평가돼 있어 이동해오는 수요 등이 증가하..
언론사: 아시아경제
썸네일: https://imgnews.pstatic.net/image/thumb100/277/2011/01/02/2522690.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0002522690&office_id=277&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=36

5

제목: [CES2011]삼성전자 "올 스마트TV 1천만대 팔 것"
작성일: 2011-01-02 10:25
요약: = 삼성전자는 올해 스마트TV 판매 목표량을 지난해 대비 두 배 이상인 1000만대 이상으로 잡았다고 2일 밝혔다. 이는 현재 전 세계 TV업계..
언론사: 뉴시스
썸네일: https://imgnews.pstatic.net/image/thumb100/003/2011/01/02/3620264.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0003620264&office_id=003&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=36

6

제목: 12월 다섯째주 코스피 개인 순매도 1위 `하이닉스`
작성일: 2011-01-02 10:05
요약: 개인 순매도 2위는 삼성전자로, 규모는 1076억원을 기록했다. 삼성생명(828억원), NHN(462억원), 엔씨소프트(459억원) 등은 5위 ..
언론사: 이데일리
썸네일: https://imgnews.pstatic.net/image/thumb100/018/2011/01/02/2365178.jpg
기사링크: https://finance.

제목: [전자·반도체 전망] 공급 넘치던 반도체업계 좋아질듯
작성일: 2011-01-01 11:47
요약: 삼성전자의 갤럭시S에 사용되는 아몰레드는 화면이 밝고 선명하다는 장점이 있지만 그동안 공급이 수요에 미치지 않아 채택률이 낮았다. 하지만 새해부..
언론사: 매일경제
썸네일: https://imgnews.pstatic.net/image/thumb100/009/2011/01/01/2382362.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0002382362&office_id=009&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=37

9

제목: “2011년 1000만원이 1억 될 종목” 묻지마 쩜상이 기대된다!
작성일: 2011-01-01 11:05
요약: 국내 최고의 기술보유로 삼성전자 등 메이저IT기업에 고부가 제품을 공급하고 있으며, 다른 IT 장비 업체들이 진입하기는 사실 불가능한 탄탄한 기술력과, 제휴를..
언론사: 파이낸셜뉴스
썸네일: https://imgnews.pstatic.net/image/thumb100/009/2011/01/01/2382362.jpg
기사링크: https://finance.naver.com/news/news_read.naver?article_id=0002378101&office_id=014&mode=search&query=%BB%EF%BC%BA%C0%FC%C0%DA&page=37

10

제목: '소득 4만불 선진국' 도약위해 중견기업 키우자
작성일: 2011-01-01 10:35
요약: ▶김종구 대표(이하 김 대표)＝삼성전자와 LG전자, 현대·기아자동차 같은 초우량 글로벌 기업들이 지속적으로 선전하기 위해서는 글로벌 경쟁력을 갖..
언론사: 머니투데이
썸네일: https://imgnews.pstatic.net/image/thumb100/008/2011/01/01/2455479.jpg
기사링크: https

In [63]:
data = {
        'ticker_code' : 005930,     → 종목 티커
        'ticker_name' : 삼성전자,     → 종목 이름
        'media_code' : 1,                  → 언론사 식별번호
        'media_name' : '서울경제',  → 언론사 이름
        'news_code' : news_code, → 뉴스 고유 코드(id)
        'news_title' : title, → 기사제목
        'news_summary' : summary, → 기사 요약내용
        'news_thumb_url' : thumbnail_url, → 기사 이미지 경로(이미지는 맨 앞에 것만)
        'url' : news_url, → 기사 URL
        'news_reg_date' : regist_date → 기사입력일
            }

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<ipython-input-63-c7c9c977fd1f>, line 2)